# When do accretors spin down (w.r.t. He core burning)
author: [Mathieu Renzo](mailto:mrenzo@flatironinstitute.org)

$\omega$ and $J_\mathrm{tot}$ profiles and evolution

In [ ]:
from MESAreader import getSrcCol, read_MESA_header
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import warnings
import paths


In [ ]:
root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/"
root_rot = root + "single_stars/Z_0.0019/"
accretors_root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/binaries/Z_0.0019/"
folders_accretors = sorted(glob.glob(accretors_root+'*/'))


In [ ]:
def plot_Jtot_time(hfile, ax, annotate = False,  **plot_kwargs):
    src, col = getSrcCol(hfile)
    Jtot = src[:, col.index("log_total_angular_momentum")]
    t = src[:, col.index("star_age")]
    tx = np.log10(t[-1]-t)
    ax.plot(tx, Jtot, **plot_kwargs)
    if annotate:
        center_h1 = src[:, col.index("center_h1")]
        center_he4 = src[:, col.index("center_he4")]
        iTAMS = np.argmin(np.absolute(center_h1-1e-4))
        iHedepl = np.argmin(np.absolute(center_he4-1e-4))
        ax.axvline(tx[iTAMS], 0, 1, ls="--", lw=2, zorder=0)
        ax.axvline(tx[iHedepl], 0, 1, ls="--", lw=2, zorder=0)


In [ ]:
fig = plt.figure(figsize=(10,10))
gs = gridspec.GridSpec(120, 150)
ax = fig.add_subplot(gs[:, :])

# accretors
for f in folders_accretors:
    # m1, m2 = get_masses(folders_accretors)
    hfile = f+'/LOGS2/history.data'
    src, col = getSrcCol(hfile)
    m2 = src[0, col.index("star_mass")]
    plot_Jtot_time(hfile, ax, label=r"$M_{2}="+f"{m2:.0f}"+"$", annotate=True)

prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]


# single stars
for f in glob.glob(root_rot+'*'):
    if ("0.5" in f):
        if ("18" in f):
            c = colors[0]
            hfile = f+"/LOGS/history.data"
            plot_Jtot_time(hfile, ax, lw=1, c=c, label=r"$\mathrm{single} 18$")
        elif ("20" in f):
            c = colors[1]
            hfile = f+"/LOGS/history.data"
            plot_Jtot_time(hfile, ax, lw=1, c=c, label=r"$\mathrm{single} 20$")
        elif ("36" in f):
            c = colors[2]
            hfile = f+"/LOGS/history.data"
            plot_Jtot_time(hfile, ax, lw=1, c=c, label=r"$\mathrm{single} 36$")
        
            
            

ax.legend()
ax.invert_xaxis()
ax.set_ylabel(r"$\log_{10}(J_\mathrm{tot}/ \mathrm{cgs\ units})$")
ax.set_xlabel(r"$\log_{10}(t_\mathrm{C\ depl} - t) \mathrm{[yr]}$")

# Internal $\omega$ profile

In [ ]:
single18 = glob.glob(root_rot+'/18*/')
single20 = glob.glob(root_rot+'/20*/')
single36 = glob.glob(root_rot+'/36*/')

In [ ]:
def get_initial_vrot(f):
    src, col = getSrcCol(f+'/LOGS/history.data')
    v_surf = src[0, col.index("surf_avg_v_rot")]
    return v_surf

for f in single18+single20+single36:
    print(f.split('/')[-2],get_initial_vrot(f))

$\omega/\omega_\mathrm{crit}=0.4$ is the closest to
$v_\mathrm{surf}\simeq300\,\mathrm{km\ s^{-1}}$ used in [Limong & Chieffi 2018](https://ui.adsabs.harvard.edu/abs/2018ApJS..237...13L/abstract)

In [ ]:
def plot_omega_M(pfile, ax, **plot_kwargs):
    src, col = getSrcCol(pfile)
    omega = src[:, col.index("omega")]
    log_omega = np.log10(omega)
    m = src[:, col.index("mass")]
    ax.plot(m, log_omega, **plot_kwargs)

    

In [ ]:
def get_R(pfile):
    src_header, col_header = read_MESA_header(pfile)
    photosphere_r = float(src_header[col_header.index("photosphere_r")])
    return photosphere_r

In [ ]:
fig = plt.figure(figsize=(10,10))
gs = gridspec.GridSpec(120, 150)
ax0 = fig.add_subplot(gs[:40, :])
ax1 = fig.add_subplot(gs[40:80, :])
ax2 = fig.add_subplot(gs[80:, :])

ax2.set_xlabel(r"$M \ [M_\odot]$")
ax1.set_ylabel(r"$\log_{10}(\omega/ \mathrm{rad\ s^{-1}})$")
ax1.set_xticklabels([])
ax0.set_xticklabels([])
axes = [ax0,ax1,ax2]

colors = plt.cm.viridis(np.linspace(0, 1, 6))

for f in folders_accretors:
    ax = axes[folders_accretors.index(f)]
    profiles = sorted(glob.glob(f+"/LOGS2/*Rsun*.data"), key=get_R)
    for p in profiles:
        photosphere_r = get_R(p)
        if (round(photosphere_r, 0) == 20.): continue
        if ax == ax0:
            plot_omega_M(p, ax, c=colors[profiles.index(p)], label=f"$R={photosphere_r:.0f}\,R_\odot$")
        else: # no legend
            plot_omega_M(p, ax, c=colors[profiles.index(p)])

# plot single stars
p18_04 = sorted(glob.glob(root_rot+'/18_rot0.4/LOGS/*Rsun*.data'), key=get_R)
for p in p18_04:
    photosphere_r = get_R(p)
    if (round(photosphere_r, 0) == 20.): continue
    plot_omega_M(p, ax0, ls='--', c=colors[p18_04.index(p)])

p20_04 = sorted(glob.glob(root_rot+'/20_rot0.4/LOGS/*Rsun*.data'), key=get_R)
for p in p20_04:
    photosphere_r = get_R(p)
    if (round(photosphere_r, 0) == 20.): continue
    plot_omega_M(p, ax1, ls='--', c=colors[p20_04.index(p)])



ax0.legend(fontsize=25, frameon=True)
ax1.set_xlim(ax2.get_xlim())
ax0.set_xlim(ax2.get_xlim())

ax1.plot(np.nan, np.nan, c='k', label=r"accretor")
ax1.plot(np.nan, np.nan, ls='--', c='k', label=r"$v_\mathrm{surf}\simeq 300\,\mathrm{km\ s^{-1}}$")
ax1.legend(fontsize=25, frameon=True, loc="upper right", handlelength=0.8, handletextpad=0.1)

# Plot He profiles

In [ ]:
def plot_he(pfile, ax, **plot_kwargs):
    src, col = getSrcCol(pfile)
    he4 = src[:, col.index("he4")]
    m = src[:, col.index("mass")]
    ax.plot(m, he4, **plot_kwargs)

In [ ]:
fig = plt.figure(figsize=(10,10))
gs = gridspec.GridSpec(120, 150)
ax0 = fig.add_subplot(gs[:40, :])
ax1 = fig.add_subplot(gs[40:80, :])
ax2 = fig.add_subplot(gs[80:, :])

ax2.set_xlabel(r"$M \ [M_\odot]$")
ax1.set_ylabel(r"$\log_{10}(\omega/ \mathrm{rad\ s^{-1}})$")
ax1.set_xticklabels([])
ax0.set_xticklabels([])
axes = [ax0,ax1,ax2]

for f in folders_accretors:
    ax = axes[folders_accretors.index(f)]
    profiles = sorted(glob.glob(f+"/LOGS2/*Rsun*.data"), key=get_R)
    colors = plt.cm.viridis(np.linspace(0, 1, len(profiles)))
    for p in profiles:
        photosphere_r = get_R(p)
        if (round(photosphere_r, 0) == 20.): continue
        plot_he(p, ax, c=colors[profiles.index(p)], label=f"$R={photosphere_r:.0f}$")
    

ax0.legend(fontsize=25, frameon=True)

ax1.set_xlim(ax2.get_xlim())
ax0.set_xlim(ax2.get_xlim())

